In [1]:
from platform import python_version
python_version()

'3.9.7'

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint

In [3]:
plugin_names = []
reviews = []
functionalities = []
ratings = []
descriptions = []
product_urls = []
tags = []
security = []
support = []
pricings = []
instals = []
web_url = "https://marketplace.atlassian.com"

In [4]:
def countRatings(item):
    rating = 0
    full_rating = item.find_all("span",class_ = "css-1hjeuno-StarIconContainer e5bskoh4")
    fraction_rating = item.find_all("span", class_ = "css-mbudn6-StarIconContainer e5bskoh4")
    rating = len(full_rating)+len(fraction_rating)*.5
    return rating

def convert(num):
    return str(num).replace(',','')

def seperate(string):
    lst = ["Jira Cloud", "Jira Server", "Jira Data Center"]
    return ','.join(list(filter(lambda x:  x in string, lst)))

In [6]:
search_url= "https://marketplace.atlassian.com/search?category=Time%20tracking&product=jira"
driver = webdriver.Chrome("/Users/lamisa/chromedriver/chromedriver")
driver.get(search_url)
content = driver.page_source
soup = BeautifulSoup(content,'html.parser')

/var/folders/6w/6h6k8r3s4fl4kqz5hb4rpkj40000gn/T/ipykernel_1759/2308961190.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/Users/lamisa/chromedriver/chromedriver")


In [ ]:
#extract title, no. of reviews, app descrption, average ratings

items = soup.find_all("a", class_ = "e13wqmfi2 css-1uhn52o-linkCss-linkCss-HitLinkStyled-HitLinkStyled e11ijshh0", href = True)
for item in items:
    if item.text:
        product_urls.append(web_url+item['href'])
        
    title = item.find("h3", class_ = "css-x53e1b-H3-baseHeadingsCss-baseHeadingsCss-h3Css-h3Css-NameHeader e1fhg9wy5")
    review = item.find("span",class_ = "css-92dv6g")
    description = item.find("div", class_ = "css-jdzn71-Body e13wqmfi4")
    ratings.append(countRatings(item))
    
    if title is not None:
        plugin_names.append(title.text)
    else:
        plugin_names.append('None')
        
    if review is not None:
        reviews.append(review.text)
    else:
        reviews.append('0')
        
    if description is not None:
        descriptions.append(description.text)
    else:
        descriptions.append('None')
    


In [ ]:
#get app_ids
app_ids = []
for url in product_urls:
    i = url.split('/')[-2]
    app_ids.append(i)
print(len(app_ids))

In [ ]:
# extract monthly flat pricing for cloud service, tags, security support, installs

for url in product_urls:
    page = url.replace("overview","pricing")
    driver = webdriver.Chrome(executable_path = "/Users/lamisa/chromedriver/chromedriver")
    driver.get(page)
    sleep(randint(2,10))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    pricing = soup.find("div", class_ = "css-1cf5srj-AppCost-baseHeadingsCss-baseHeadingsCss eu58ut81")
    tag = soup.find("div",class_ = "css-8oat5o-CompatibilitiesContainer e1x0fti24")
    sc = soup.find("span", class_= "css-1pyq0a2-BadgeLabel-CharlieFontFamily-CharlieFontFamily e1t4w93o3")
    sp = soup.find("a", class_ = "css-96dsix-LinkStyled e1x0fti211")
    install = soup.find("span", class_ = "css-1y9uv3s-InstallCountText e5w14hq2")
    
    if install is not None:
        instals.append(install.text.split(' ')[0])
    else:
        instals.append('0')
    if tag is not None:
        tags.append(tag.text)
    else:
        tags.append("None")
    if sc is not None:
        security.append(sc.text)
    else:
        security.append("None")
    if sp is not None:
        support.append(sp.text)
    else:
        support.append("None")
    if pricing is not None:
        pricings.append([url.split('/')[-2],pricing.text])
    else:
        pricings.append([url.split('/')[-2],'0'])


In [ ]:
#extract monthly per_user pricing for cloud service
monthly_per_user_pricing=[]

for url in product_urls:
    page = url.replace("overview","pricing")
    driver = webdriver.Chrome(executable_path = "/Users/lamisa/chromedriver/chromedriver")
    driver.get(page)
    sleep(randint(2,10))
    try:
        driver.find_element_by_class_name("css-la31wz").send_keys(Keys.BACKSPACE*2+'99')
    except:
        print('None')
    #driver.find_element_by_id("").click()
    sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    peru_pricing = soup.find("div", class_ = "css-1m2viiz-AppCostDescription eu58ut83")
    if peru_pricing is not None:
        monthly_per_user_pricing.append(peru_pricing.text)
    else:
        monthly_per_user_pricing.append('0')
    driver.close()

    

In [ ]:
#take monthly flat price (10 people)
monthly_flat_price = []

for pricing in pricings:
    #print(pricing[1])
    monthly_flat_price.append(str(pricing[1]).replace('USD',''))

In [ ]:
#print((new_pricings))
print(len(monthly_flat_price))

In [ ]:
new_df = pd.DataFrame({'App_id':app_ids,'Name':plugin_names,'Monthly_Flat_price':monthly_flat_price,"Monthly_per_user_price":monthly_per_user_pricing,'Installs':instals,'Reviews':reviews,'Avg Rating':ratings, 'Service Type':tags, 'Security':security, 'Own By': support, 'Description':descriptions,'Product_urls': product_urls}) 
new_df.Installs = new_df.Installs.astype('string')
new_df['Installs'] = new_df['Installs'].apply(lambda x: convert(x))
new_df['Service Type'] = new_df['Service Type'].apply(lambda x: seperate(x))

In [ ]:
new_df.to_csv('products.csv', index=False, encoding='utf-8')